### Pandas vs Dask vs DuckDB

In [9]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import duckdb
import polars as pl
import pyarrow

### Create csv file

In [10]:
pd.DataFrame({
               'A': np.random.randint(0, 100, size=20000000),
               'B': np.random.randint(0, 100, size=20000000),
               'C': np.random.randint(0, 100, size=20000000),
              }).to_csv('dataset.csv', index=False)

### Pandas read csv file

In [20]:
%%time
df = pd.read_csv('dataset.csv', engine='pyarrow')
print(f"Pandas: shape = {df.shape}")

Pandas: shape = (20000000, 3)
CPU times: user 2.37 s, sys: 876 ms, total: 3.24 s
Wall time: 325 ms


### Dask read csv file

In [19]:
%%time
dask_df = dd.read_csv('dataset.csv')
print(f"Dask: shape = {dask_df.compute().shape}")

Dask: shape = (20000000, 3)
CPU times: user 1.49 s, sys: 432 ms, total: 1.92 s
Wall time: 1.09 s


### Polars read csv file

In [21]:
%%time
polars_df = pl.read_csv('dataset.csv')
print(f"Polars: shape = {polars_df.shape}")

Polars: shape = (20000000, 3)
CPU times: user 1.99 s, sys: 482 ms, total: 2.48 s
Wall time: 248 ms


In [22]:
%%time
# duckdb.read_csv('dataset.csv')
duckdb.query("""SELECT * FROM read_csv('dataset.csv');""")

CPU times: user 23.1 ms, sys: 40.4 ms, total: 63.5 ms
Wall time: 640 ms


┌────────┬───────┬───────┐
│   A    │   B   │   C   │
│ int64  │ int64 │ int64 │
├────────┼───────┼───────┤
│     23 │    98 │    61 │
│     35 │    19 │    97 │
│      0 │    22 │    67 │
│     47 │    52 │    95 │
│     17 │    72 │    90 │
│     91 │    22 │     3 │
│     17 │    44 │    61 │
│     19 │    95 │    19 │
│     59 │    50 │    29 │
│     71 │    77 │    96 │
│      · │     · │     · │
│      · │     · │     · │
│      · │     · │     · │
│     66 │    17 │    72 │
│     59 │    60 │    33 │
│      0 │    37 │    52 │
│      8 │    63 │    88 │
│     84 │    18 │    88 │
│     31 │    19 │    84 │
│     90 │    29 │    85 │
│     63 │    74 │    59 │
│      8 │    92 │    24 │
│     76 │    14 │    57 │
├────────┴───────┴───────┤
│ ? rows       3 columns │
└────────────────────────┘

In [14]:
%%time
pandas_grouped = df.groupby(['A', 'B']).agg({'C': 'sum'})
pandas_grouped.head()

CPU times: user 501 ms, sys: 180 ms, total: 681 ms
Wall time: 800 ms


C
A B        
0 0  100760
  1   96378
  2   98186
  3  100660
  4  101088

In [15]:
%%time
dask_groupby = dask_df.groupby(['A', 'B']).agg({'C': 'sum'})
dask_df.head()

CPU times: user 807 ms, sys: 136 ms, total: 943 ms
Wall time: 969 ms


,A,B,C
0,23,98,61
1,35,19,97
2,0,22,67
3,47,52,95
4,17,72,90


In [16]:
%%time
duckdb.query("""
             SELECT A, B, SUM(C) FROM read_csv('dataset.csv')
             group by A, B
             order by A, B
             """)

CPU times: user 10.4 ms, sys: 23.9 ms, total: 34.2 ms
Wall time: 34.8 ms


┌───────┬───────┬────────┐
│   A   │   B   │ sum(C) │
│ int64 │ int64 │ int128 │
├───────┼───────┼────────┤
│     0 │     0 │ 100760 │
│     0 │     1 │  96378 │
│     0 │     2 │  98186 │
│     0 │     3 │ 100660 │
│     0 │     4 │ 101088 │
│     0 │     5 │  99860 │
│     0 │     6 │  92448 │
│     0 │     7 │ 104719 │
│     0 │     8 │  97470 │
│     0 │     9 │  99622 │
│     · │     · │    ·   │
│     · │     · │    ·   │
│     · │     · │    ·   │
│    99 │    90 │  99002 │
│    99 │    91 │  99224 │
│    99 │    92 │  98911 │
│    99 │    93 │  99956 │
│    99 │    94 │ 103089 │
│    99 │    95 │ 102644 │
│    99 │    96 │  93856 │
│    99 │    97 │  95969 │
│    99 │    98 │  99628 │
│    99 │    99 │  98471 │
├───────┴───────┴────────┤
│ ? rows       3 columns │
└────────────────────────┘

In [17]:
%%time
# crashing ....
# merged_pandas = pd.merge(df, df)

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.96 μs


In [18]:
%%time
# # crashing ....
# merged_dask = dd.merge(dask_df, dask_df)
# print(f"Dask: shape = {merged_dask.compute().shape}")

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.05 μs


In [30]:
%%time
selected_pandas = df[df['A'] > 60]
# selected_pandas.shape
selected_pandas.head()

CPU times: user 179 ms, sys: 100 ms, total: 279 ms
Wall time: 276 ms


pandas.core.series.Series

In [20]:
%%time
selected_dask = dask_df[dask_df['A'] > 60]
selected_dask.compute().shape

CPU times: user 1.9 s, sys: 455 ms, total: 2.36 s
Wall time: 1.26 s


(7802093, 3)

In [21]:
%%time
duckdb.query("""
             SELECT A, B FROM read_csv('dataset.csv')
             where A > 60
             """)

CPU times: user 6.61 ms, sys: 24.3 ms, total: 30.9 ms
Wall time: 31.5 ms


┌────────────────┬───────┐
│       A        │   B   │
│     int64      │ int64 │
├────────────────┼───────┤
│             91 │    22 │
│             71 │    77 │
│             86 │    12 │
│             94 │    57 │
│             80 │    41 │
│             82 │    70 │
│             95 │    51 │
│             74 │    92 │
│             76 │    32 │
│             72 │     7 │
│              · │     · │
│              · │     · │
│              · │     · │
│             63 │    29 │
│             85 │    68 │
│             68 │     2 │
│             77 │     7 │
│             76 │    66 │
│             66 │    17 │
│             86 │     7 │
│             68 │    72 │
│             76 │    70 │
│             99 │    88 │
├────────────────┴───────┤
│ ? rows       2 columns │
└────────────────────────┘

In [22]:
def my_function(x):
    return x * 2

In [23]:
%%time
applied_pandas = df['A'].apply(my_function)
applied_pandas.head()

CPU times: user 4.59 s, sys: 172 ms, total: 4.76 s
Wall time: 4.76 s


0    46
1    70
2     0
3    94
4    34
Name: A, dtype: int64

In [24]:
def calc_vec(narr, multiplier):
    narr *= multiplier

In [25]:
%%time
# vectorize_pandas = (df['A'] * 2)
# vectorize_pandas.head()
calc_vec(df['A'], 2)
df.head()

CPU times: user 46.5 ms, sys: 20.2 ms, total: 66.8 ms
Wall time: 66.3 ms


,A,B,C
0,46,98,61
1,70,19,97
2,0,22,67
3,94,52,95
4,34,72,90


In [26]:
%%time
applied_dask = dask_df['A'].map(my_function)
applied_dask.head()

/media/zensei/My Passport/Zensei/Projects/Python/financial_data/.venv/lib/python3.11/site-packages/dask_expr/_collection.py:4165: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('A', 'int64'))

  warnings.warn(meta_warning(meta))


CPU times: user 586 ms, sys: 87.8 ms, total: 674 ms
Wall time: 725 ms


0    46
1    70
2     0
3    94
4    34
Name: A, dtype: int64

In [27]:
%%time
duckdb.query("""
             SELECT A * 2 as double_up FROM read_csv('dataset.csv')
             """)

CPU times: user 7.44 ms, sys: 32.2 ms, total: 39.6 ms
Wall time: 86.7 ms


┌────────────────────────┐
│       double_up        │
│         int64          │
├────────────────────────┤
│                     46 │
│                     70 │
│                      0 │
│                     94 │
│                     34 │
│                    182 │
│                     34 │
│                     38 │
│                    118 │
│                    142 │
│                     ·  │
│                     ·  │
│                     ·  │
│                    132 │
│                    118 │
│                      0 │
│                     16 │
│                    168 │
│                     62 │
│                    180 │
│                    126 │
│                     16 │
│                    152 │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘